Note: I left the code to generate the htfs for each test system in the notebook, but if you don't want to re-generate, you can use the pickled htf that I saved (I left the full path in for you). I generated these htfs based after you added support for `lambda_{i}_bonds_old/new` and `scale_lambda_0_bonds`

In [1]:
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools
from perses.tests.test_topology_proposal import generate_dipeptide_top_pos_sys, generate_atp
import pickle
import copy

#############################################
# CONSTANTS
#############################################
temperature = 298.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CUDA")


conducting subsequent work with the following platform: CUDA


INFO:rdkit:Enabling RDKit 2021.03.1 jupyter extensions


In [2]:
def test_angle_energies(htf, is_old=True, is_solvated=False, check_scale=False):
    htf_copy = copy.deepcopy(htf)
    
    # Get harmonic angle force and old/new positions
    system = htf._topology_proposal.old_system if is_old else htf._topology_proposal.new_system
    harmonic_angle_force = system.getForce(1) 
    positions = htf.old_positions(htf.hybrid_positions) if is_old else htf.new_positions(htf.hybrid_positions)
    
    # Get custom angle force and hybrid positions
    angle_force_index = 2 if is_solvated else 1
    hybrid_system = htf.hybrid_system
    custom_angle_force = hybrid_system.getForce(angle_force_index)
    hybrid_positions = htf.hybrid_positions
    
    # Set global parameters
    lambda_old = 1 if is_old else 0
    lambda_new = 0 if is_old else 1
    for i in range(custom_angle_force.getNumGlobalParameters()):
        if custom_angle_force.getGlobalParameterName(i) == 'lambda_0_angles_old':
            custom_angle_force.setGlobalParameterDefaultValue(i, lambda_old)
        if custom_angle_force.getGlobalParameterName(i) == 'lambda_0_angles_new':
            custom_angle_force.setGlobalParameterDefaultValue(i, lambda_new)

    # Zero the unique old/new angles in the custom bond force
    hybrid_to_angle_indices = htf._hybrid_to_new_angle_indices if is_old else htf._hybrid_to_old_angle_indices
    for hybrid_idx, idx in hybrid_to_angle_indices.items():
        p1, p2, p3, hybrid_params = custom_angle_force.getAngleParameters(hybrid_idx)
        hybrid_params = list(hybrid_params)
        index_to_zero = -3 if is_old else -1
        hybrid_params[index_to_zero] *= 0
        custom_angle_force.setAngleParameters(hybrid_idx, p1, p2, p3, hybrid_params)

    ## Get energy components of standard bond force
    platform = configure_platform(REFERENCE_PLATFORM)
    thermostate_other = ThermodynamicState(system=system, temperature=temperature)
    integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
    context_other = thermostate_other.create_context(integrator_other)
    context_other.setPositions(positions)
    components_other = compute_potential_components(context_other, beta=beta)
    print(components_other)
    
    ## Get energy components of custom bond force
    if is_solvated:
        custom_angle_force.setUsesPeriodicBoundaryConditions(True)
    thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
    integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
    context_hybrid.setPositions(hybrid_positions)
    components_hybrid = compute_potential_components(context_hybrid, beta=beta)
    print(components_hybrid)
    
    assert np.isclose([components_other[1][1]], [components_hybrid[1][1]])
    
    print("Success! Custom angle force and standard angle force energies are equal!")
    
    if check_scale:
        
        # Get custom bond force and hybrid positions
        angle_force_index = 2 if is_solvated else 1
        hybrid_system = htf_copy.hybrid_system
        custom_angle_force = hybrid_system.getForce(angle_force_index)
        hybrid_positions = htf_copy.hybrid_positions
        
        if is_solvated:
            custom_angle_force.setUsesPeriodicBoundaryConditions(True)
        
        ## Get energy components of custom bond force
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid = compute_potential_components(context_hybrid, beta=beta)
        print(components_hybrid)
        
        # Set `scale_lambda_{i}` to 0.5
        for i in range(custom_angle_force.getNumGlobalParameters()):
            if custom_angle_force.getGlobalParameterName(i) == 'scale_lambda_0_angles':
                custom_angle_force.setGlobalParameterDefaultValue(i, 0.5)
            elif custom_angle_force.getGlobalParameterName(i) == 'interscale_lambda_0_angles':
                custom_angle_force.setGlobalParameterDefaultValue(i, 0.5)
        
        ## Get energy components of custom bond force iwth scaling
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid_scaled = compute_potential_components(context_hybrid, beta=beta)
        print(components_hybrid_scaled)
        
        assert not np.isclose([components_hybrid[1][1]], [components_hybrid_scaled[1][1]])
        
        print("Success! Scaling the angle force changes the energy")

# Alanine dipeptide in vacuum

In [3]:
# Create a htf
atp, system_generator = generate_atp(phase = 'vacuum')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions
INFO:geometry:There are 8 new atoms
INFO:geometry:	reduced angle potential = 0.5875389405036744.


conducting subsequent work with the following platform: CUDA


INFO:geometry:	reduced angle potential = 0.1040183279948801.
INFO:geometry:	reduced angle potential = 0.11051116822991956.
INFO:geometry:	reduced angle potential = 0.22654933803102464.
INFO:geometry:	reduced angle potential = 1.321428883859808.
INFO:geometry:	reduced angle potential = 0.7536237659613949.
INFO:geometry:	reduced angle potential = 1.194245588583566.
INFO:geometry:	reduced angle potential = 0.06663844039091607.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angle forces in the no-nonbonded final system
INFO:geometry:	there are 72 torsion forces in the no-nonbonded

...

INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986687678176
INFO:geometry:final reduced energy 29.78946596595154
INFO:geometry:sum of energies: 29.789467829390215


conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463344085812), ('CustomBondForce', 12.970883364040223)]


INFO:geometry:magnitude of difference in the energies: 1.8634386762528266e-06
INFO:geometry:Final logp_proposal: -26802.882152574442
INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda'], ['nonscale_region']]
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=n

In [4]:
# Save htf
with open("atp_vacuum.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [5]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [6]:
htf.hybrid_system.getForces()

[<simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b6b83325240> >,
 <simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2b6b826deed0> >,
 <simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2b6bd3e833c0> >]

In [11]:
test_angle_energies(htf)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', -39.4499277456003), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.9500065360888446), ('CustomAngleForce', 0.6112775462128995), ('CustomTorsionForce', 42.212441468407036), ('AndersenThermostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [12]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [14]:
test_angle_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 2.9494828408599925), ('HarmonicAngleForce', 6.182663028922969), ('PeriodicTorsionForce', 25.9270338060766), ('NonbondedForce', -51.32400448116353), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.9500065360888446), ('CustomAngleForce', 6.182663028922968), ('CustomTorsionForce', 42.212441468407036), ('AndersenThermostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, one scale region

In [25]:
# Create a htf
atp, system_generator = generate_atp(phase = 'vacuum')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True, 
                                scale_regions=[[10, 11, 12, 13, 22, 23, 24, 25, 26, 27, 28, 29]])

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions
INFO:geometry:There are 8 new atoms
INFO:geometry:	reduced angle potential = 0.019230952894899275.


conducting subsequent work with the following platform: CUDA


INFO:geometry:	reduced angle potential = 0.00017107204611475823.
INFO:geometry:	reduced angle potential = 1.0073598863007964.
INFO:geometry:	reduced angle potential = 0.785754564681716.
INFO:geometry:	reduced angle potential = 0.008690943101301947.
INFO:geometry:	reduced angle potential = 0.3014180148793759.
INFO:geometry:	reduced angle potential = 2.285468184266583.
INFO:geometry:	reduced angle potential = 0.47599995095707076.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angle forces in the no-nonbonded final system
INFO:geometry:	there are 72 torsion forces in the no-nonbo

...

INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986687678176
INFO:geometry:final reduced energy 29.78946596595154
INFO:geometry:sum of energies: 29.789467829390215


conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463344085812), ('CustomBondForce', 12.970883364040223)]


INFO:geometry:magnitude of difference in the energies: 1.8634386762528266e-06
INFO:geometry:Final logp_proposal: -26796.910836891406
INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda', 'scale_lambda_0', 'interscale_lambda_0'], ['nonscale_region', 'scale_region_0', 'interscale_region_0']]
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPer

In [26]:
# Save htf
with open("atp_vacuum_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [27]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [28]:
htf.hybrid_system.getForces()

[<simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b6be1b396c0> >,
 <simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2b6be1b397b0> >,
 <simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2b6bdc3bc2a0> >]

In [29]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[10, 11, 12, 13, 22, 23, 24, 25, 26, 27, 28, 29]

In [30]:
test_angle_energies(htf)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', -39.4499277456003), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.693266216004257), ('CustomAngleForce', 0.6112775462128995), ('CustomTorsionForce', 44.25025233594987), ('AndersenThermostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [31]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [32]:
test_angle_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 1.6927425207754043), ('HarmonicAngleForce', 7.286038243853887), ('PeriodicTorsionForce', 27.964844243582032), ('NonbondedForce', 26.243705122449896), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.693266216004257), ('CustomAngleForce', 7.286038243853889), ('CustomTorsionForce', 44.25025233594987), ('AndersenThermostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


# Alanine dipeptide in solvent

In [3]:
# Create a htf
atp, system_generator = generate_atp(phase = 'solvent')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [10, 14, 18, 16, 17, 15, 13, 19]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching fun

generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions
INFO:geometry:There are 8 new atoms
INFO:geometry:	reduced angle potential = 0.6030216606353751.


...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986672125617
INFO:geometry:final reduced energy 29.78946596595156
INFO:geometry:sum of energies: 29.789467813837657
INFO:geometry:magnitude of difference in the energies: 1.8478860965842614e-06
INFO:geometry:Final logp_proposal: -26840.97686061595


conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463328533253), ('CustomBondForce', 12.970883364040223)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda'], ['nonscale_region']]
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.56477354, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.56477354, z=0.0), unit=nanometer), Quantity

( nonscale_lambda_bonds * nonscale_region ) * (K/2)*(r-length)^2;K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;


In [4]:
# Save htf
with open("atp_solvent.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [5]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [6]:
test_angle_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.034783653565214685), ('HarmonicAngleForce', 0.6112775462129015), ('PeriodicTorsionForce', 16.285398289983082), ('NonbondedForce', -917.909292205925), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.8319751057620108), ('CustomAngleForce', 0.6112775462128995), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [7]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [8]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 1.831451410533151), ('HarmonicAngleForce', 7.55641047986655), ('PeriodicTorsionForce', 28.332649521292527), ('NonbondedForce', -464.38846644797746), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.8319751057620108), ('CustomAngleForce', 7.556410479866574), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, one scale region

In [9]:
# Create a htf
atp, system_generator = generate_atp(phase = 'solvent')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True, 
                                scale_regions=[[10, 11, 12, 13, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564]])

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [10, 14, 18, 13, 17, 19, 16, 15]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


generating geometry engine
making geometry proposal from ALA to THR


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1654 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (7, 6, 8, 10, [1, Quantity(value=0.99000

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986672125617
INFO:geometry:final reduced energy 29.78946596595156
INFO:geometry:sum of energies: 29.789467813837657
INFO:geometry:magnitude of difference in the energies: 1.8478860965842614e-06
INFO:geometry:Final logp_proposal: -27019.174262826844


conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463328533253), ('CustomBondForce', 12.970883364040223)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda', 'scale_lambda_0', 'interscale_lambda_0'], ['nonscale_region', 'scale_region_0', 'interscale_region_0']]
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.56477354, y=0.0, z=0.0), unit=nano

( nonscale_lambda_bonds * nonscale_region+scale_lambda_0_bonds * scale_region_0+interscale_lambda_0_bonds * interscale_region_0 ) * (K/2)*(r-length)^2;K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;


In [10]:
# Save htf
with open("atp_solvent_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [11]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [12]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[10, 11, 12, 13, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564]

In [13]:
test_angle_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.034783653565214685), ('HarmonicAngleForce', 0.6112775462129015), ('PeriodicTorsionForce', 16.285398289983082), ('NonbondedForce', -998.7187278370578), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.1311292704376184), ('CustomAngleForce', 0.6112775462128995), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [14]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [15]:
test_angle_energies(htf, is_old=False, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 1.1306055752088155), ('HarmonicAngleForce', 6.637643065078332), ('PeriodicTorsionForce', 27.196819628622556), ('NonbondedForce', -291.0292057841858), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.1311292704376184), ('CustomAngleForce', 6.637643065078238), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.1311292704376184), ('CustomAngleForce', 7.09179049020389), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.1311292704376184), ('CustomAngleForce', 3.6244603056455693), ('AndersenThermostat', 0.0)

# CDK2 transformation in solvent

In [47]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


In [61]:
# Create a htf
setup_directory = resource_filename("perses", "data/cdk2-example")

# Get options
from perses.app.setup_relative_calculation import getSetupOptions
yaml_filename = os.path.join(setup_directory, "cdk2_setup_repex.yaml")
setup_options = getSetupOptions(yaml_filename)

# Update options
for parameter in ['protein_pdb', 'ligand_file']:
    setup_options[parameter] = os.path.join(setup_directory, setup_options[parameter])
    
setup_options['rxn_field'] = True
setup_options['phases'] = ['solvent']
setup_options['scale_regions'] = None
setup_options['validate_endstate_energies'] = False
    
setup_dict = setup_relative_calculation.run_setup(setup_options, serialize_systems=False, build_samplers=False)

2021-05-04 18:06:08,378:(5.12s):root:	Detecting phases...
2021-05-04 18:06:08,382:(0.00s):root:		phases detected: ['complex', 'solvent', 'vacuum']
2021-05-04 18:06:08,385:(0.00s):root:No constraints will be removed
2021-05-04 18:06:08,388:(0.00s):root:No spectators
2021-05-04 18:06:08,391:(0.00s):root:			run_type is not specified; default to None
2021-05-04 18:06:08,392:(0.00s):root:	Detecting fe_type...
2021-05-04 18:06:08,393:(0.00s):root:		fe_type: repex
2021-05-04 18:06:08,394:(0.00s):root:			offline-freq not specified: default to 10.
2021-05-04 18:06:08,401:(0.01s):root:	'neglect_angles' detected: False.
2021-05-04 18:06:08,402:(0.00s):root:	'softcore_v2' not specified: default to 'False'
2021-05-04 18:06:08,403:(0.00s):root:	Creating 'output'...
2021-05-04 18:06:08,412:(0.01s):root:	detected ligand file: /home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf
2021-05-04 18:06:08,413:(0.00s):ro

2021-05-04 18:06:11,950:(0.00s):geometry:Conducting forward proposal...
2021-05-04 18:06:11,952:(0.00s):geometry:Computing proposal order with NetworkX...
2021-05-04 18:06:11,961:(0.01s):geometry:number of atoms to be placed: 1
2021-05-04 18:06:11,964:(0.00s):geometry:Atom index proposal order is [2190]
2021-05-04 18:06:11,965:(0.00s):geometry:omitted_bonds: []
2021-05-04 18:06:11,967:(0.00s):geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
2021-05-04 18:06:12,166:(0.20s):geometry:creating growth system...
2021-05-04 18:06:12,224:(0.06s):geometry:	creating bond force...
2021-05-04 18:06:12,227:(0.00s):geometry:	there are 27 bonds in reference force.
2021-05-04 18:06:12,229:(0.00s):geometry:	creating angle force...
2021-05-04 18:06:12,230:(0.00s):geometry:	there are 85 angles in reference force.
2021-05-04 18:06:12,233:(0.00s):geometry:	creating torsion force...
2021-05-04 18:06:12,235:(0.00s):geometry:	creating extra

conducting subsequent work with the following platform: CUDA


2021-05-04 18:06:12,768:(0.46s):geometry:setting atoms_with_positions context new positions
2021-05-04 18:06:13,135:(0.37s):geometry:There are 1 new atoms


conducting subsequent work with the following platform: CUDA


...

2021-05-04 18:06:16,633:(0.11s):geometry:	beginning construction of no_nonbonded final system...
2021-05-04 18:06:16,635:(0.00s):geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
2021-05-04 18:06:16,755:(0.12s):geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
2021-05-04 18:06:16,757:(0.00s):geometry:	there are 28 bond forces in the no-nonbonded final system
2021-05-04 18:06:16,759:(0.00s):geometry:	there are 85 angle forces in the no-nonbonded final system
2021-05-04 18:06:16,760:(0.00s):geometry:	there are 157 torsion forces in the no-nonbonded final system
2021-05-04 18:06:16,762:(0.00s):geometry:reverse final system defined with 0 neglected angles.


conducting subsequent work with the following platform: CUDA


2021-05-04 18:06:17,617:(0.86s):geometry:total reduced potential before atom placement: 48.41899607369698


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.0879842954280575), ('CustomAngleForce', 0.04605409749261916), ('CustomTorsionForce', 0.005423562569706604), ('CustomBondForce', -5.1962986311777435)]


2021-05-04 18:06:18,684:(1.07s):geometry:total reduced energy added from growth system: -5.056836675687361
2021-05-04 18:06:18,686:(0.00s):geometry:final reduced energy 43.36216011253515
2021-05-04 18:06:18,688:(0.00s):geometry:sum of energies: 43.362159398009624
2021-05-04 18:06:18,690:(0.00s):geometry:magnitude of difference in the energies: 7.145255302987152e-07
2021-05-04 18:06:18,691:(0.00s):geometry:Final logp_proposal: 11.810926163885583
2021-05-04 18:06:18,817:(0.13s):root:	writing pickle output...
2021-05-04 18:06:19,211:(0.39s):root:	successfully dumped pickle.
2021-05-04 18:06:19,215:(0.00s):root:	setup is complete.  Writing proposals and positions for each phase to top_prop dict...
2021-05-04 18:06:19,217:(0.00s):root:	writing atom_mapping.png
2021-05-04 18:06:19,267:(0.05s):root:	 steps per move application: 50
2021-05-04 18:06:19,270:(0.00s):root:	trajectory prefix: out
2021-05-04 18:06:19,271:(0.00s):root:	atom selection detected: not water
2021-05-04 18:06:19,273:(0.00s

In [62]:
setup_options

{'protein_pdb': '/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_fixed_nohet.pdb',
 'ligand_file': '/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf',
 'old_ligand_index': 14,
 'new_ligand_index': 15,
 'forcefield_files': ['amber/protein.ff14SB.xml',
  'amber/tip3p_standard.xml',
  'amber/tip3p_HFE_multivalent.xml'],
 'small_molecule_forcefield': 'gaff-2.11',
 'small_molecule_parameters_cache': 'cdk2-cache.json',
 'pressure': 1.0,
 'temperature': 300.0,
 'solvent_padding': 9.0,
 'save_setup_pickle_as': 'fesetup_hbonds.pkl',
 'n_steps_per_move_application': 50,
 'fe_type': 'repex',
 'checkpoint_interval': 250,
 'n_cycles': 2,
 'n_states': 6,
 'n_equilibration_iterations': 0,
 'trajectory_directory': 'output',
 'trajectory_prefix': 'out',
 'atom_selection': 'not water',
 'phases': ['solvent'],
 'protocol-type': 

In [63]:
htf = setup_dict['hybrid_topology_factories']['solvent']

In [64]:
# Save htf
with open("cdk2_solvent.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [67]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [68]:
test_angle_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954756), ('HarmonicAngleForce', 29.199523609209102), ('PeriodicTorsionForce', 15.076747992617829), ('NonbondedForce', 1620.3489177881142), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.199523609209102), ('CustomTorsionForce', 30.189918794847536), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [70]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [71]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899664), ('HarmonicAngleForce', 33.00516953842139), ('PeriodicTorsionForce', 15.113155351932875), ('NonbondedForce', 1636.7942547439848), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 33.00516953842139), ('CustomTorsionForce', 30.189918794847536), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, one scale region

In [72]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle

In [73]:
# Create a htf
setup_directory = resource_filename("perses", "data/cdk2-example")

# Get options
from perses.app.setup_relative_calculation import getSetupOptions
yaml_filename = os.path.join(setup_directory, "cdk2_setup_repex.yaml")
setup_options = getSetupOptions(yaml_filename)

# Update options
for parameter in ['protein_pdb', 'ligand_file']:
    setup_options[parameter] = os.path.join(setup_directory, setup_options[parameter])
    
setup_options['rxn_field'] = True
setup_options['phases'] = ['solvent']
setup_options['scale_regions'] = [[44, 2197]]
setup_options['validate_endstate_energies'] = False
    
setup_dict = setup_relative_calculation.run_setup(setup_options, serialize_systems=False, build_samplers=False)

2021-05-04 18:07:14,979:(54.05s):root:	Detecting phases...
2021-05-04 18:07:14,982:(0.00s):root:		phases detected: ['complex', 'solvent', 'vacuum']
2021-05-04 18:07:14,984:(0.00s):root:No constraints will be removed
2021-05-04 18:07:14,986:(0.00s):root:No spectators
2021-05-04 18:07:14,988:(0.00s):root:			run_type is not specified; default to None
2021-05-04 18:07:14,990:(0.00s):root:	Detecting fe_type...
2021-05-04 18:07:14,992:(0.00s):root:		fe_type: repex
2021-05-04 18:07:14,994:(0.00s):root:			offline-freq not specified: default to 10.
2021-05-04 18:07:14,995:(0.00s):root:	'neglect_angles' detected: False.
2021-05-04 18:07:14,997:(0.00s):root:	'softcore_v2' not specified: default to 'False'
2021-05-04 18:07:14,999:(0.00s):root:	Creating 'output'...
2021-05-04 18:07:15,003:(0.00s):root:	detected ligand file: /home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf
2021-05-04 18:07:15,006:(0.00s):r

2021-05-04 18:07:18,525:(0.00s):geometry:Conducting forward proposal...
2021-05-04 18:07:18,527:(0.00s):geometry:Computing proposal order with NetworkX...
2021-05-04 18:07:18,537:(0.01s):geometry:number of atoms to be placed: 1
2021-05-04 18:07:18,539:(0.00s):geometry:Atom index proposal order is [2190]
2021-05-04 18:07:18,541:(0.00s):geometry:omitted_bonds: []
2021-05-04 18:07:18,542:(0.00s):geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
2021-05-04 18:07:18,723:(0.18s):geometry:creating growth system...
2021-05-04 18:07:18,770:(0.05s):geometry:	creating bond force...
2021-05-04 18:07:18,772:(0.00s):geometry:	there are 27 bonds in reference force.
2021-05-04 18:07:18,774:(0.00s):geometry:	creating angle force...
2021-05-04 18:07:18,776:(0.00s):geometry:	there are 85 angles in reference force.
2021-05-04 18:07:18,778:(0.00s):geometry:	creating torsion force...
2021-05-04 18:07:18,780:(0.00s):geometry:	creating extra

conducting subsequent work with the following platform: CUDA


2021-05-04 18:07:19,342:(0.49s):geometry:setting atoms_with_positions context new positions
2021-05-04 18:07:19,683:(0.34s):geometry:There are 1 new atoms
2021-05-04 18:07:19,691:(0.01s):geometry:	reduced angle potential = 0.2448361016944241.


conducting subsequent work with the following platform: CUDA


...

2021-05-04 18:07:23,197:(0.11s):geometry:	beginning construction of no_nonbonded final system...
2021-05-04 18:07:23,199:(0.00s):geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
2021-05-04 18:07:23,320:(0.12s):geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
2021-05-04 18:07:23,322:(0.00s):geometry:	there are 28 bond forces in the no-nonbonded final system
2021-05-04 18:07:23,324:(0.00s):geometry:	there are 85 angle forces in the no-nonbonded final system
2021-05-04 18:07:23,326:(0.00s):geometry:	there are 157 torsion forces in the no-nonbonded final system
2021-05-04 18:07:23,327:(0.00s):geometry:reverse final system defined with 0 neglected angles.


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-05-04 18:07:24,204:(0.88s):geometry:total reduced potential before atom placement: 48.41899607369698


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.0879842954280575), ('CustomAngleForce', 0.04605409749261916), ('CustomTorsionForce', 0.005423562569706604), ('CustomBondForce', -5.1962986311777435)]


2021-05-04 18:07:25,252:(1.05s):geometry:total reduced energy added from growth system: -5.056836675687361
2021-05-04 18:07:25,254:(0.00s):geometry:final reduced energy 43.36216011253515
2021-05-04 18:07:25,256:(0.00s):geometry:sum of energies: 43.362159398009624
2021-05-04 18:07:25,257:(0.00s):geometry:magnitude of difference in the energies: 7.145255302987152e-07
2021-05-04 18:07:25,259:(0.00s):geometry:Final logp_proposal: 11.565896514368898
2021-05-04 18:07:25,386:(0.13s):root:	writing pickle output...
2021-05-04 18:07:25,537:(0.15s):root:	successfully dumped pickle.
2021-05-04 18:07:25,541:(0.00s):root:	setup is complete.  Writing proposals and positions for each phase to top_prop dict...
2021-05-04 18:07:25,543:(0.00s):root:	writing atom_mapping.png
2021-05-04 18:07:25,593:(0.05s):root:	 steps per move application: 50
2021-05-04 18:07:25,595:(0.00s):root:	trajectory prefix: out
2021-05-04 18:07:25,597:(0.00s):root:	atom selection detected: not water
2021-05-04 18:07:25,599:(0.00s

In [56]:
setup_options

{'protein_pdb': '/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_fixed_nohet.pdb',
 'ligand_file': '/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf',
 'old_ligand_index': 14,
 'new_ligand_index': 15,
 'forcefield_files': ['amber/protein.ff14SB.xml',
  'amber/tip3p_standard.xml',
  'amber/tip3p_HFE_multivalent.xml'],
 'small_molecule_forcefield': 'gaff-2.11',
 'small_molecule_parameters_cache': 'cdk2-cache.json',
 'pressure': 1.0,
 'temperature': 300.0,
 'solvent_padding': 9.0,
 'save_setup_pickle_as': 'fesetup_hbonds.pkl',
 'n_steps_per_move_application': 50,
 'fe_type': 'repex',
 'checkpoint_interval': 250,
 'n_cycles': 2,
 'n_states': 6,
 'n_equilibration_iterations': 0,
 'trajectory_directory': 'output',
 'trajectory_prefix': 'out',
 'atom_selection': 'not water',
 'phases': ['solvent'],
 'protocol-type': 

In [74]:
htf = setup_dict['hybrid_topology_factories']['solvent']

In [75]:
# Save htf
with open("cdk2_solvent_scale_regions.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [76]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent_scale_regions.pickle", "rb") as f:
    htf = pickle.load(f)

In [77]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[44, 2197]

In [79]:
test_angle_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954756), ('HarmonicAngleForce', 29.199523609209102), ('PeriodicTorsionForce', 15.076747992617829), ('NonbondedForce', 1624.6215250399368), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.199523609209102), ('CustomTorsionForce', 30.15410291939743), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [93]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent_scale_regions.pickle", "rb") as f:
    htf = pickle.load(f)

In [94]:
test_angle_energies(htf, is_old=False, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899664), ('HarmonicAngleForce', 29.724810973822063), ('PeriodicTorsionForce', 15.077341017914472), ('NonbondedForce', 1639.6900550143098), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.724810973822063), ('CustomTorsionForce', 30.15410291939743), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.771174159217313), ('CustomTorsionForce', 30.15410291939743), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.6

# 8mer in solvent

In [16]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle
from simtk import openmm, unit
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [28]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [29]:
test_angle_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.349099420288431), ('HarmonicAngleForce', 12.529213143555554), ('PeriodicTorsionForce', 157.292123218685), ('NonbondedForce', -10459.718829031748), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.349099420288431), ('CustomAngleForce', 12.529213143555554), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [19]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)


In [20]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.3031685822946475), ('HarmonicAngleForce', 306.5325531504853), ('PeriodicTorsionForce', 152.3548522619778), ('NonbondedForce', -10210.139290660007), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.349099420288431), ('CustomAngleForce', 306.53255315048534), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, one scale region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [30]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [22]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[21, 23, 24, 25, 26, 5576, 5575]

In [31]:
test_angle_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.349099420288431), ('HarmonicAngleForce', 12.529213143555554), ('PeriodicTorsionForce', 157.292123218685), ('NonbondedForce', -10255.444378993296), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.349099420288431), ('CustomAngleForce', 12.529213143555554), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.349099420288431), ('CustomAngleForce', 161.18843543403023), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.349099420288431), ('CustomAngleForce', 86.73664557221115), ('AndersenThermostat', 0.0), 

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [32]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [33]:
test_angle_energies(htf, is_old=False, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.3031685822946475), ('HarmonicAngleForce', 160.94407800086674), ('PeriodicTorsionForce', 152.18268170083624), ('NonbondedForce', -10122.58254369068), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.349099420288431), ('CustomAngleForce', 160.94407800086674), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.349099420288431), ('CustomAngleForce', 161.18843543403023), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 7.349099420288431), ('CustomAngleForce', 86.73664557221115), ('AndersenThermostat', 0.0)

## Barnase:barstar (old htf, no naked charge fix or counterion fix)

### Test one alchemical region, no scale regions

#### Old system HarmonicAngleForce vs hybrid system CustomAngleForce

In [3]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [4]:
test_angle_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 190.9699139958041), ('HarmonicAngleForce', 797.0922707362615), ('PeriodicTorsionForce', 3856.2346108037013), ('NonbondedForce', -25366.343173297213), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 190.97001343592), ('CustomAngleForce', 797.0922707362635), ('CustomTorsionForce', 3856.234842190186), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicAngleForce vs hybrid system CustomAngleForce

In [5]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [6]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 190.55887687012904), ('HarmonicAngleForce', 982.3171572386327), ('PeriodicTorsionForce', 3849.684097413479), ('NonbondedForce', -24973.072364300806), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 190.97001343592), ('CustomAngleForce', 982.3171572386327), ('CustomTorsionForce', 3856.234842190186), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, no scale regions

#### Old system HarmonicAngleForce vs hybrid system CustomAngleForce

In [8]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [10]:
test_angle_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 190.96991399580065), ('HarmonicAngleForce', 797.0922707362635), ('PeriodicTorsionForce', 3856.2346108037004), ('NonbondedForce', -25367.423468547222), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 191.68441661118993), ('CustomAngleForce', 797.0922707362635), ('CustomTorsionForce', 3856.234842190186), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 191.68441661118993), ('CustomAngleForce', 1002.3374923455682), ('CustomTorsionForce', 3856.234842190186), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1

#### New system HarmonicAngleForce vs hybrid system CustomAngleForce

In [11]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [12]:
test_angle_energies(htf, is_old=False, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 191.27328004539896), ('HarmonicAngleForce', 1000.7045263915769), ('PeriodicTorsionForce', 3849.683473240853), ('NonbondedForce', -24737.764251905406), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 191.68441661118993), ('CustomAngleForce', 1000.7045263915769), ('CustomTorsionForce', 3856.234842190186), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 191.68441661118993), ('CustomAngleForce', 1002.3374923455682), ('CustomTorsionForce', 3856.234842190186), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 

In [ ]:
# TO DO: actually test the scaling of the energies matches